In [ ]:
#autoreload
%load_ext autoreload
%autoreload 2

import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

import sys
sys.path.append("../../src_clasificacion_vistas/common")

import pycimg
from matplotlib.backends.backend_pdf import PdfPages


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 # Catalogo Defectos
 
 El objeto de este Notebook es generar un pdf por tipo de defecto con vistas en las que está anotado dicho defecto, con el fin de tener una idea global del aspecto del defecto

In [ ]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_patch_reddish/train_scores.json"
val_results_json="../out_evaluate_patch_reddish/val_scores.json"
train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    




def create_view_tables(train_data,clases):
    labels=[]
    preds=[]
    imags=[]
    filenames=[]
    for d in train_data:

        filenamejson=d['filename'].replace(".npz",".json")
        scores=d['probs_vistas']
        kk=os.path.join("..",filenamejson)
        npzfilename=kk.replace(".json",".npz")
        imagenes=pycimg.npzread_torch(npzfilename,kk,channel_list=['R','G','B'])
        if not os.path.exists(kk):
            print(" does not exist: ",kk)
            continue
        with open(kk, "r") as f:
            anot_dict = json.load(f)

        if 'views_annotations' not in anot_dict:
            print("No hay anotaciones de vistas en el json ",filenamejson)
            continue    
        truth_vistas=anot_dict['views_annotations']
        
        nvistas=len(truth_vistas)
        nclases=len(clases)
        truths=[]
        
        for n in range(nvistas):
            keyid="view_"+str(n)
            truths.append(truth_vistas[keyid])
        for vista in imagenes:
            imags.append(vista.permute(1,2,0)[:,:,:3])
            
        gt=torch.zeros((nvistas,nclases))    
        
        
        # truths: lista de vistas. Cada una de ellas es un diccionario con el nombre del defecto como clave
        for n in range(nvistas):
            for m, defecto in enumerate(clases):
                if defecto in truths[n]:
                    gt[n,m]=float(truths[n][defecto])
                    
        preds1=torch.zeros((nvistas,nclases))
        for n in range(nvistas):
            for m, defecto in enumerate(clases):
                if defecto in scores:
                    preds1[n,m]=float(scores[defecto][n])
        print(gt.shape,preds1.shape)
        labels.append(gt)
        preds.append(preds1)
        filenames+=[filenamejson]*nvistas
            
    labels=torch.concat(labels,axis=0)
    preds=torch.concat(preds,axis=0)

    return preds,labels,filenames,imags



with open(train_results_json, "r") as f:
    train_dict = json.load(f)

with open(val_results_json, "r") as f:
    val_dict = json.load(f)    

train_data=train_dict['train_results']
val_data = val_dict['val_results']
clases=list(train_dict['train_results'][0]['ground_truth'].keys())

print(len(train_data))
print(len(val_data))

1320
352


In [8]:
preds_train,labels_train,filenames_train,imags_train=create_view_tables(train_data,clases)
preds_val,labels_val,filenames_val,imags_val=create_view_tables(val_data,clases)
print(len(filenames_train),len (imags_train))
print(preds_train.shape,labels_train.shape)

torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([8, 13]) torch.Size([8, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13]) torch.Size([11, 13])
torch.Size([11, 13

# Cantidad de Vistas de con cada defecto

In [9]:
print("==========================================")
print("  ** VAL **")
print("==========================================")

n_por_defecto=labels_val.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_val.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_val.shape[0])

print("\n==========================================")
print("  ** TRAIN **")
print("==========================================")

n_por_defecto=labels_train.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_train.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_train.shape[0])

  ** VAL **
cracks : 52
damage_by_pliers : 39
deformed_peduncle : 35
green : 128
hollow : 96
light_scars : 275
oil_spots : 63
plu_stickers : 75
reddish : 45
rotten : 125
scars : 180
skin_breakdown : 137
surface : 711
N sin defectos: 1641
N total: 3350

  ** TRAIN **
cracks : 101
damage_by_pliers : 156
deformed_peduncle : 246
green : 605
hollow : 269
light_scars : 1001
oil_spots : 298
plu_stickers : 371
reddish : 177
rotten : 533
scars : 704
skin_breakdown : 456
surface : 2432
N sin defectos: 6245
N total: 12723


## Crear los pdfs

In [10]:
# preds_train,labels_train,filenames_train,imags_train
# preds_val,labels_val,filenames_val,imags_val

preds_all=np.concatenate((preds_train,preds_val),axis=0)
labels_all=np.concatenate((labels_train,labels_val),axis=0)
filenames_all=filenames_train+filenames_val
imags_all=imags_train+imags_val

In [16]:
np.argwhere(labels_all[:,0]>0.5)

array([[ 2679],
       [ 2680],
       [ 2890],
       [ 2891],
       [ 3251],
       [ 3844],
       [ 3998],
       [ 3999],
       [ 4221],
       [ 4222],
       [ 4229],
       [ 4230],
       [ 4790],
       [ 4791],
       [ 4883],
       [ 4884],
       [ 4885],
       [ 4891],
       [ 5237],
       [ 5238],
       [ 5245],
       [ 5246],
       [ 5247],
       [ 5266],
       [ 5267],
       [ 5268],
       [ 5269],
       [ 5421],
       [ 5958],
       [ 5965],
       [ 5966],
       [ 5967],
       [ 5968],
       [ 5996],
       [ 6003],
       [ 7929],
       [ 7930],
       [ 7931],
       [ 7932],
       [ 7941],
       [ 7942],
       [ 7943],
       [ 7944],
       [ 7945],
       [ 7946],
       [ 7947],
       [ 7948],
       [ 7949],
       [ 7950],
       [ 7951],
       [ 7957],
       [ 7958],
       [ 7959],
       [ 7960],
       [ 7961],
       [ 7963],
       [ 7970],
       [ 7971],
       [ 7972],
       [ 7973],
       [ 7978],
       [ 7979],
       [

In [ ]:
max_num_pages=300
for c,clase in enumerate(clases):
    casos_positivos=np.argwhere(labels_all[:,c]>0.5)[:,0]
    pdfname="../defect_catalog/samples_"+clase+".pdf"
    if len(casos_positivos)>max_num_pages:
        casos_positivos=np.random.choice(casos_positivos,max_num_pages,replace=False)
    print(clase," N casos positivos: ",len(casos_positivos))    
    with PdfPages(pdfname) as pdf :
        for i in casos_positivos:
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(imags_all[i])
            #ax[0].set_title(filenames_all[i])
            ax[0].axis('off')
            ax[1].bar(np.arange(len(clases)), preds_all[i])
            #ax[1].set_title(f'Predictions for {clase}')
            ax[1].set_xticks(np.arange(len(clases)))
            ax[1].set_xticklabels(clases, rotation=90)
            ax[1].set_ylim(0, 1)
            pdf.savefig(fig)
            plt.close(fig)
    print("Saved ",pdfname)



cracks  N casos positivos:  153
Saved  ../defect_catalog/samples_cracks.pdf
damage_by_pliers  N casos positivos:  195
Saved  ../defect_catalog/samples_damage_by_pliers.pdf
deformed_peduncle  N casos positivos:  281
Saved  ../defect_catalog/samples_deformed_peduncle.pdf
green  N casos positivos:  300
Saved  ../defect_catalog/samples_green.pdf
hollow  N casos positivos:  300
Saved  ../defect_catalog/samples_hollow.pdf
light_scars  N casos positivos:  300
Saved  ../defect_catalog/samples_light_scars.pdf
oil_spots  N casos positivos:  300


KeyboardInterrupt: 